In [10]:
%pip install keras
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 228.5 MB 75 kB/s  eta 0:00:01    |██████                          | 42.9 MB 6.1 MB/s eta 0:00:31     |███████▏                        | 51.3 MB 266 kB/s eta 0:11:06     |█████████▎                      | 66.0 MB 2.0 MB/s eta 0:01:22     |██████████▎                     | 73.3 MB 122 kB/s eta 0:21:04     |████████████                    | 86.1 MB 1.3 MB/s eta 0:01:51     |███████████████████▌            | 139.2 MB 1.3 MB/s eta 0:01:11     |███████████████████▋            | 140.2 MB 296 kB/s eta 0:04:58     |████████████████████▍           | 145.3 MB 1.1 MB/s eta 0:01:17     |██████████████████████          | 156.7 MB 1.9 MB/s eta 0:00:39     |████████████████████████████▋   | 204.3 MB 1.2 MB/s eta 0:00:21
     |████████████████████████████████| 438 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 126 

In [38]:
#imports
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tolgayasar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
df =  pd.read_csv('../data/processed/preprocessed-data.csv', encoding = 'ISO-8859-1', index_col=0)

def remove_trash(text, trash_list):
    for trash in trash_list:
        text = text.replace(trash, ' ')
    return text
df['Summary'] = df['Summary'].map(lambda x: remove_trash(x,['\n', '&quot;','&#39;']))
ratings_with_summary = df[ (df['Summary']!= '9')]

books_with_summary = ratings_with_summary.groupby('isbn').apply(lambda df: df.iloc[0]).loc[:,['isbn', 'book_title', 'Summary']]
display(books_with_summary["Summary"])

isbn
0007110928    Biography of Billy Connolly, written by his wi...
0020442203    The best-selling rack edition of The Lion, the...
002542730X    A whimsical adaptation of classic fairy tales ...
0060008776    Father Terry Dunn thought he d seen everything...
006001203X    With Dr. Atkins  New Diet Revolution you get e...
                                    ...                        
3548359698    Die Autoren werfen einen amÃÂ¼santen Blick au...
3596150655    Ein aus Erinnerungen, Analysen und Impressione...
8445071416    Sacado de su comodo agujero-hobbit por Gandalf...
8807813025    Il libro racchiude la storia, raccontata dall ...
8845205118    An aviator whose plane is forced down in the S...
Name: Summary, Length: 1400, dtype: object

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [40]:
documents_df=pd.DataFrame(books_with_summary["Summary"].to_numpy(),columns=['documents'])

stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [49]:
documents_df['documents_cleaned'].to_numpy()

array(['biography billy connolly  written wife pamela stephenson ',
       'best selling rack edition lion  witch wardrobe movie still cover eight page movie still insert  excellent homeschool use',
       'whimsical adaptation classic fairy tales bedtime stories removes kinds bias objective language traditional tales chicken little  rapunzel  three little pigs  cinderella  many others ',
       ...,
       'sacado de su comodo agujero hobbit por gandalf una banda de enanos  bilbo se encuentra de pronto en medio de una conspiracion que pretende apoderarse del tesoro de smaug el magnifico  un enorme muy peligroso dragon ',
       'il libro racchiude la storia  raccontata dall amico suonatore di tromba  sotto forma di monologo  di danny boodmann t  d  lemon novecento  pianista sul transatlantico virginian ',
       'aviator whose plane forced sahara desert encounters little man small planet describes adventures universe seeking secret really important life '],
      dtype=object)

In [50]:
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models.tfidfmodel import TfidfModel
from gensim.parsing.preprocessing import preprocess_documents, preprocess_string
from gensim.corpora import Dictionary

text_corpus_preprocessed = preprocess_documents(documents_df['documents_cleaned'].to_numpy())
text_corpus_preprocessed

[['biographi', 'billi', 'connolli', 'written', 'wife', 'pamela', 'stephenson'],
 ['best',
  'sell',
  'rack',
  'edit',
  'lion',
  'witch',
  'wardrob',
  'movi',
  'cover',
  'page',
  'movi',
  'insert',
  'excel',
  'homeschool',
  'us'],
 ['whimsic',
  'adapt',
  'classic',
  'fairi',
  'tale',
  'bedtim',
  'stori',
  'remov',
  'kind',
  'bia',
  'object',
  'languag',
  'tradit',
  'tale',
  'chicken',
  'littl',
  'rapunzel',
  'littl',
  'pig',
  'cinderella'],
 ['father',
  'terri',
  'dunn',
  'thought',
  'seen',
  'mean',
  'street',
  'detroit',
  'went',
  'littl',
  'retreat',
  'rwanda',
  'evad',
  'tax',
  'fraud',
  'indict'],
 ['atkin',
  'new',
  'diet',
  'revolut',
  'need',
  'essenti',
  'medic',
  'nutrit',
  'inform',
  'help',
  'carbohydr',
  'gram',
  'counter',
  'low',
  'carb',
  'meal',
  'plan',
  'dozen',
  'mouth',
  'water',
  'recip',
  'appet'],
 ['februari', 'fifteenth', 'special', 'dai'],
 ['spellbind',
  'novel',
  'unfold',
  'draw',
  'rea

In [55]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=text_corpus_preprocessed, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
vector = model.wv['love']  # get numpy vector of a word
sims = model.wv.most_similar('love', topn=10)  # get other similar words
sims

[('find', 0.8947703838348389),
 ('investig', 0.8814330697059631),
 ('turn', 0.8809208273887634),
 ('year', 0.877566397190094),
 ('murder', 0.8722959160804749),
 ('stori', 0.8712796568870544),
 ('famili', 0.8701426386833191),
 ('novel', 0.8654875159263611),
 ('life', 0.8631676435470581),
 ('woman', 0.8626711964607239)]

In [ ]:
tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

In [41]:
# tokenize and pad every document to make them of the same size
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')

vocab_size=len(tokenizer.word_index)+1
print (tokenized_paded_documents[456])

[ 521 2869  167  875    5  104  421   30  422 5058  198 2965  652 5059
  590 5060 2966 2967 5061 2968   81 2969    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [31]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
import gensim.downloader
#W2V_PATH="GoogleNews-vectors-negative300.bin.gz"
#model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)
model_w2v = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [42]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index.
embedding_matrix=np.zeros((vocab_size,300))

for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i]=model_w2v[word]

# creating document-word embeddings
document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))

for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]

document_word_embeddings.shape

(1400, 64, 300)

In [43]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents_df.iloc[ix]["documents"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

In [45]:
# calculating average of word vectors of a document weighted by tf-idf
document_embeddings=np.zeros((len(tokenized_paded_documents),300))
words=tfidfvectoriser.get_feature_names()

for i in range(len(document_word_embeddings)):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]

print (document_embeddings.shape)
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)
most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')

/Users/tolgayasar/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: dimension mismatch